# Examen Final: Pronóstico de propagación de pandemia en EE.UU. usando modelos de series de tiempo

## Objetivo

Este notebook tiene como objetivo predecir la **propagación de la pandemia de COVID-19** en los estados de EE.UU., basándose en los datos de carga viral en aguas residuales. Usaremos modelos de series de tiempo como **LSTM** y **Transformer** para hacer predicciones sobre la evolución futura de la pandemia, con un enfoque de propagación entre estados utilizando una **matriz de adyacencia**.

### Fase 1: Definición del problema
1. **Predicción por Estado:** Entrenaremos un modelo generalizado usando datos de un estado (inicialmente Nueva York) y lo aplicaremos a todos los estados.
2. **Umbral de Riesgo:** Definiremos un umbral para identificar qué estados están en riesgo de entrar en una pandemia.
3. **Propagación a Estados Adyacentes:** Usaremos una matriz de adyacencia para simular cómo los estados vecinos influencian la propagación del virus.
4. **Modelo de Propagación Ajustado:** Entrenaremos el modelo ajustado para incluir el efecto de la proximidad geográfica entre los estados.

### Fase 2: Implementación técnica
1. Descargaremos los datos de **CDC Wastewater** utilizando la API del **National Wastewater Surveillance System (NWSS)**.
2. Limpieza y exploración de los datos, convirtiéndolos en una serie de tiempo adecuada para entrenamiento.
3. Entrenamiento de los modelos **LSTM** y **Transformer** para hacer predicciones de la carga viral.
4. Evaluación del rendimiento del modelo usando **MAE, RMSE y MAPE**.
5. Pronóstico para los próximos **30 días** y propagación entre estados adyacentes.

Notebook plantilla con estructura lista para:
- Construir serie de carga viral en aguas residuales (CDC NWSS).
- Hacer split temporal train/val/test sin fuga.
- Escalar solo con train y aplicar a val/test.
- Crear ventanas deslizantes por split.
- Entrenar LSTM, comparar vs baseline y hacer pronóstico futuro.
- Extender a propagación entre estados con matriz de adyacencia (TODO).


## 1. Imports, configuración de paths y seeds


In [63]:
import os
import random
from pathlib import Path
from typing import Tuple, Dict, List

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam    
from tensorflow.keras.layers import (
    Input,
    MultiHeadAttention,
    LayerNormalization,
    GlobalAveragePooling1D,
    Dense,
    Dropout,
)
from tensorflow.keras import Model
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Lambda

# Paths base
PROJECT_ROOT = Path('.')
DATA_DIR = PROJECT_ROOT / 'data'
LANDING_DIR = DATA_DIR / 'landing'
PROCESSED_DIR = DATA_DIR / 'processed'
FIGURES_DIR = PROJECT_ROOT / 'figures'
MODELS_DIR = PROJECT_ROOT / 'models'

for d in (DATA_DIR, LANDING_DIR, PROCESSED_DIR, FIGURES_DIR, MODELS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Seed global
GLOBAL_SEED = 63

def set_global_seed(seed: int = 42):
    global GLOBAL_SEED
    GLOBAL_SEED = seed
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_global_seed(GLOBAL_SEED)
print(f'Seed global fijada en {GLOBAL_SEED}')
print('PROJECT_ROOT:', PROJECT_ROOT.resolve())


Seed global fijada en 63
PROJECT_ROOT: C:\Users\esteb\apps\Wastewater-SARS-CoV-2


## 2. Helpers: ventanas, métricas, gráficas y forecasting


In [64]:
def create_sliding_windows(
    series: np.ndarray,
    window_size: int,
    horizon: int = 1,
    stride: int = 1,
) -> Tuple[np.ndarray, np.ndarray]:
    """Crea ventanas deslizantes sobre una serie ya escalada o cruda.
    Se asume que se usa con un split (train / val / test) a la vez.
    """
    series = np.asarray(series).astype(float)
    T = len(series)
    if T < window_size + horizon:
        raise ValueError('Serie demasiado corta para el window_size y horizon dados.')

    X, y = [], []
    last_start = T - window_size - horizon + 1
    for start in range(0, last_start, stride):
        end = start + window_size
        target_end = end + horizon
        X.append(series[start:end])
        y.append(series[end:target_end])

    X = np.stack(X)
    y = np.stack(y)
    return X, y


def regression_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    eps = 1e-8
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), eps))) * 100
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}


def plot_time_series(dates, values, title: str = '', ylabel: str = ''):
    dates = pd.to_datetime(dates)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=values, mode='lines+markers', name='serie'))
    fig.update_layout(title=title, xaxis_title='Fecha', yaxis_title=ylabel, template='plotly_white')
    return fig


def plot_history_vs_pred(dates, y_true, y_pred, title: str = 'Histórico vs predicción'):
    dates = pd.to_datetime(dates)
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=y_true, mode='lines', name='Real'))
    fig.add_trace(go.Scatter(x=dates, y=y_pred, mode='lines', name='Predicho', line=dict(dash='dash')))
    fig.update_layout(title=title, xaxis_title='Fecha', yaxis_title='Valor', template='plotly_white')
    return fig


def plot_future_forecast(history_dates, history_values, future_dates, future_preds, title: str = 'Pronóstico futuro'):
    history_dates = pd.to_datetime(history_dates)
    future_dates = pd.to_datetime(future_dates)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history_dates, y=history_values, mode='lines', name='Histórico'))
    fig.add_trace(go.Scatter(x=future_dates, y=future_preds, mode='lines+markers', name='Pronóstico', line=dict(dash='dash')))
    fig.update_layout(title=title, xaxis_title='Fecha', yaxis_title='Valor', template='plotly_white')
    return fig


def default_predict_fn(model, X):
    if hasattr(model, 'predict'):
        return model.predict(X)
    return model(X)


def recursive_forecast(model, last_window: np.ndarray, n_future: int, predict_fn=default_predict_fn) -> np.ndarray:
    """Pronóstico recursivo n_future pasos usando modelo one-step-ahead (en espacio escalado)."""
    window = np.asarray(last_window).reshape(1, -1)
    preds = []
    for _ in range(n_future):
        y_hat = predict_fn(model, window)
        y_hat = np.asarray(y_hat).reshape(-1)
        y_next = float(y_hat[0])
        preds.append(y_next)
        window = np.roll(window, -1, axis=1)
        window[0, -1] = y_next
    return np.array(preds)

def lstm_predict_fn(model, X_2d: np.ndarray):
    """
    X_2d: shape (batch, window_size)
    Lo convertimos a (batch, window_size, 1) para el LSTM.
    """
    X_2d = np.asarray(X_2d)
    X_3d = X_2d.reshape(X_2d.shape[0], X_2d.shape[1], 1)
    return model.predict(X_3d)

def transformer_predict_fn(model, X_2d: np.ndarray):
    """
    X_2d: shape (batch, window_size)
    Lo inflamos a (batch, window_size, 1) para el Transformer.
    """
    X_2d = np.asarray(X_2d)
    X_3d = X_2d.reshape(X_2d.shape[0], X_2d.shape[1], 1)
    return model.predict(X_3d)

def get_sinusoidal_positional_encoding(max_len: int, d_model: int) -> np.ndarray:
    """
    Positional encoding tipo 'Attention is All You Need'.

    Devuelve un array de shape (1, max_len, d_model) listo para sumarse a (batch, T, d_model).
    """
    positions = np.arange(max_len)[:, np.newaxis]          # (T, 1)
    dims = np.arange(d_model)[np.newaxis, :]               # (1, D)

    angle_rates = 1.0 / np.power(10000, (2 * (dims // 2)) / d_model)
    angle_rads = positions * angle_rates                   # (T, D)

    pe = np.zeros_like(angle_rads)
    # índices pares → senos
    pe[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # índices impares → cosenos
    pe[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pe = pe[np.newaxis, ...]  # (1, T, D)
    return pe.astype("float32")

print('Helpers cargados.')

Helpers cargados.


## 3. Descarga de datos NWSS (CDC)


In [65]:
CDC_WASTEWATER_CSV_URL = 'https://data.cdc.gov/api/views/j9g8-acpt/rows.csv?accessType=DOWNLOAD'
CDC_WASTEWATER_FULL_CSV_PATH = LANDING_DIR / 'cdc_wastewater_sarscov2_full.csv'

def download_cdc_wastewater_full(url: str = CDC_WASTEWATER_CSV_URL,
                                 out_path: Path = CDC_WASTEWATER_FULL_CSV_PATH,
                                 use_cache: bool = True) -> Path:
    out_path = Path(out_path)
    if use_cache and out_path.exists():
        print(f'[INFO] Usando archivo en caché: {out_path}')
        return out_path
    print(f'[INFO] Descargando datos NWSS desde {url}')
    r = requests.get(url)
    r.raise_for_status()
    out_path.write_bytes(r.content)
    print(f'[OK] Guardado en {out_path}')
    return out_path

# download_cdc_wastewater_full()


## 4. Carga y exploración rápida


In [66]:
df_raw = pd.read_csv(CDC_WASTEWATER_FULL_CSV_PATH)
print('Shape bruto:', df_raw.shape)
display(df_raw.head())
print(df_raw.columns.tolist())

C:\Users\esteb\AppData\Local\Temp\ipykernel_49224\4203612754.py:1: DtypeWarning:

Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.



Shape bruto: (504369, 35)


,sewershed_id,wwtp_jurisdiction,county_fips,counties_served,population_served,sample_id,sample_collect_date,sample_type,sample_matrix,sample_location,...,pcr_target_flowpop_lin,pcr_target_mic_lin,hum_frac_target_mic,hum_frac_mic_conc,hum_frac_mic_unit,rec_eff_percent,rec_eff_target_name,rec_eff_spike_matrix,rec_eff_spike_conc,date_updated
0,711,me,23019,Penobscot,2500,000516f8c0f05102d4a010b987f62273,2023-11-28,24-hr flow-weighted composite,raw wastewater,wwtp,...,3.634599e+08,0.02864,pepper mild mottle virus,1.002597e+07,copies/l wastewater,68.74511,brsv vaccine,raw sample post pasteurization,5.34357,09/26/2025 10:40:00 AM
1,1809,ri,44007,Providence,10000,040ce1a855db659d046911c5d5758314,2023-07-05,24-hr time-weighted composite,raw wastewater,wwtp,...,4.267105e+07,0.00092,pepper mild mottle virus,1.772809e+08,copies/l wastewater,80.15343,brsv vaccine,raw sample post pasteurization,5.00000,09/26/2025 10:40:00 AM
2,322,fl,12115,Sarasota,100000,052760ee8f2bec3e7e4ac25f5bff23b4,2023-08-14,24-hr flow-weighted composite,post grit removal,wwtp,...,1.263573e+08,0.00455,pepper mild mottle virus,1.163784e+08,copies/l wastewater,20.63618,brsv vaccine,raw sample post pasteurization,5.34357,09/26/2025 10:40:00 AM
3,524,in,18113,Noble,10000,0dd046c819c8214f02eb79056e57978a,2023-12-18,24-hr time-weighted composite,raw wastewater,wwtp,...,5.170796e+07,0.00177,pmmov (gt-digital),4.752000e+07,copies/l wastewater,33.02887,bcov vaccine,raw sample,5.45000,09/26/2025 10:40:00 AM
4,694,me,23001,Androscoggin,60000,0e08cd627f3702430558aaf38aefa6e4,2023-09-13,24-hr flow-weighted composite,raw wastewater,wwtp,...,1.066697e+08,0.01578,pepper mild mottle virus,1.410123e+07,copies/l wastewater,6.26729,brsv vaccine,raw sample post pasteurization,5.34357,09/26/2025 10:40:00 AM


['sewershed_id', 'wwtp_jurisdiction', 'county_fips', 'counties_served', 'population_served', 'sample_id', 'sample_collect_date', 'sample_type', 'sample_matrix', 'sample_location', 'flow_rate', 'concentration_method', 'pasteurized', 'pcr_type', 'extraction_method', 'major_lab_method', 'inhibition_detect', 'inhibition_adjust', 'ntc_amplify', 'pcr_target', 'pcr_gene_target_agg', 'pcr_target_avg_conc', 'pcr_target_units', 'lod_sewage', 'pcr_target_avg_conc_lin', 'pcr_target_flowpop_lin', 'pcr_target_mic_lin', 'hum_frac_target_mic', 'hum_frac_mic_conc', 'hum_frac_mic_unit', 'rec_eff_percent', 'rec_eff_target_name', 'rec_eff_spike_matrix', 'rec_eff_spike_conc', 'date_updated']


## 5. Preprocesamiento base y construcción de una serie (ej. estado NY)

Se elige un estado base (ej. NY) y se agrega la carga viral por fecha usando la mediana.


In [67]:
DATE_COL = 'sample_collect_date'
TARGET_COL = 'pcr_target_flowpop_lin'
STATE_COL = 'wwtp_jurisdiction'

df_raw[DATE_COL] = pd.to_datetime(df_raw[DATE_COL], errors='coerce')
df = df_raw.dropna(subset=[DATE_COL, TARGET_COL]).copy()

STATE = 'ny'  # puedes cambiar el estado base

df_state = df.query('wwtp_jurisdiction == @STATE')[[DATE_COL, TARGET_COL]].copy()
df_state = (
    df_state
    .groupby(DATE_COL, as_index=False)[TARGET_COL]
    .median()
    .sort_values(DATE_COL)
)
df_state = df_state.rename(columns={DATE_COL: 'date', TARGET_COL: 'target'})
df_state = df_state.dropna(subset=['date', 'target'])

print('Estado base:', STATE)
print('Rango de fechas:', df_state['date'].min(), '→', df_state['date'].max())
print('N observaciones:', len(df_state))
display(df_state.head())

fig = plot_time_series(df_state['date'], df_state['target'],
                       title=f'Serie – estado {STATE} (flow/pop)', ylabel='target')
fig.show()

Estado base: ny
Rango de fechas: 2020-08-31 00:00:00 → 2025-09-17 00:00:00
N observaciones: 1366


,date,target
0,2020-08-31,2.832473e+06
1,2020-09-02,5.965088e+02
2,2020-09-04,2.355929e+04
3,2020-09-06,6.226106e+02
4,2020-09-08,5.880741e+06


## 6. Split temporal + escalado (sin fuga de información)

Primero se hace el split temporal sobre la serie cruda. Luego se ajusta el scaler con train y se aplica a val/test.


In [68]:
TRAIN_FRAC = 0.7
VAL_FRAC = 0.15
TEST_FRAC = 1.0 - TRAIN_FRAC - VAL_FRAC

SCALER_TYPE = 'minmax'  # minmax o standard

series = df_state.set_index('date')['target'].astype(float).sort_index()
n = len(series)
n_train = int(n * TRAIN_FRAC)
n_val = int(n * VAL_FRAC)
n_test = n - n_train - n_val

train_series = series.iloc[:n_train]
val_series   = series.iloc[n_train:n_train + n_val]
test_series  = series.iloc[n_train + n_val:]

print('Sizes → train:', len(train_series), 'val:', len(val_series), 'test:', len(test_series))

if SCALER_TYPE == 'minmax':
    scaler = MinMaxScaler()
elif SCALER_TYPE == 'standard':
    scaler = StandardScaler()
else:
    raise ValueError('SCALER_TYPE debe ser minmax o standard')

train_scaled = scaler.fit_transform(train_series.values.reshape(-1, 1)).reshape(-1)
val_scaled   = scaler.transform(val_series.values.reshape(-1, 1)).reshape(-1)
test_scaled  = scaler.transform(test_series.values.reshape(-1, 1)).reshape(-1)

print('Rango escalado train → min:', float(train_scaled.min()), 'max:', float(train_scaled.max()))


Sizes → train: 956 val: 204 test: 206
Rango escalado train → min: 0.0 max: 0.9999999999999998


## 7. Ventanas deslizantes por split (ya escalados)


In [69]:
WINDOW_SIZE = 7
HORIZON = 1

X_train, y_train = create_sliding_windows(train_scaled, WINDOW_SIZE, HORIZON)
X_val,   y_val   = create_sliding_windows(val_scaled,   WINDOW_SIZE, HORIZON)
X_test,  y_test  = create_sliding_windows(test_scaled,  WINDOW_SIZE, HORIZON)

print('X_train:', X_train.shape, 'y_train:', y_train.shape)
print('X_val  :', X_val.shape,   'y_val  :', y_val.shape)
print('X_test :', X_test.shape,  'y_test :', y_test.shape)

train_dates = train_series.index[WINDOW_SIZE:WINDOW_SIZE + len(y_train)]
val_dates   = val_series.index[WINDOW_SIZE:WINDOW_SIZE + len(y_val)]
test_dates  = test_series.index[WINDOW_SIZE:WINDOW_SIZE + len(y_test)]

print('Rango fechas train ventanas:', train_dates[0], '→', train_dates[-1])
print('Rango fechas test ventanas :', test_dates[0],  '→', test_dates[-1])


X_train: (949, 7) y_train: (949, 1)
X_val  : (197, 7) y_val  : (197, 1)
X_test : (199, 7) y_test : (199, 1)
Rango fechas train ventanas: 2020-09-13 00:00:00 → 2024-05-15 00:00:00
Rango fechas test ventanas : 2025-01-15 00:00:00 → 2025-09-17 00:00:00


## 8. Modelo baseline ingenuo (último valor)


In [70]:
class NaiveLastValueModel:
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        X = np.asarray(X)
        return X[:, -1:].copy()

baseline = NaiveLastValueModel().fit(X_train, y_train)

y_test_pred_baseline_scaled = baseline.predict(X_test).reshape(-1)
y_test_true_scaled = y_test.reshape(-1)

y_test_true = scaler.inverse_transform(y_test_true_scaled.reshape(-1, 1)).reshape(-1)
y_test_pred_baseline = scaler.inverse_transform(y_test_pred_baseline_scaled.reshape(-1, 1)).reshape(-1)

metrics_base = regression_metrics(y_test_true, y_test_pred_baseline)
print('Baseline – métricas test:', metrics_base)

fig = plot_history_vs_pred(test_dates, y_test_true, y_test_pred_baseline,
                           title='Baseline - Test vs predicción')
fig.show()

Baseline – métricas test: {'MAE': 8566050.49316392, 'RMSE': 20151293.6795827, 'MAPE': 319.5636773371569}


## 9. Modelo LSTM 


### 9.1 Model

In [71]:
X_train_lstm = X_train.reshape(-1, WINDOW_SIZE, 1)
X_val_lstm   = X_val.reshape(-1, WINDOW_SIZE, 1)
X_test_lstm  = X_test.reshape(-1, WINDOW_SIZE, 1)

model_lstm = Sequential([
    LSTM(120, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
    LSTM(60, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
    LSTM(30, input_shape=(WINDOW_SIZE, 1), return_sequences=False),
    # Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(HORIZON),
])

model_lstm.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
model_lstm.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 7, 120)            58560     
                                                                 
 lstm_7 (LSTM)               (None, 7, 60)             43440     
                                                                 
 lstm_8 (LSTM)               (None, 30)                10920     
                                                                 
 dense_4 (Dense)             (None, 32)                992       
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 113,945
Trainable params: 113,945
Non-trainable params: 0
_________________________________________________________________


In [72]:
EPOCHS = 200
BATCH_SIZE = 64

history_lstm = model_lstm.fit(
    X_train_lstm,
    y_train,
    validation_data=(X_val_lstm, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
)

Epoch 1/200
15/15 [==============================] - 7s 106ms/step - loss: 0.0033 - mae: 0.0228 - val_loss: 7.8164e-04 - val_mae: 0.0232
Epoch 2/200
15/15 [==============================] - 0s 25ms/step - loss: 0.0031 - mae: 0.0212 - val_loss: 4.3516e-04 - val_mae: 0.0102
Epoch 3/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0030 - mae: 0.0205 - val_loss: 4.4338e-04 - val_mae: 0.0070
Epoch 4/200
15/15 [==============================] - 0s 26ms/step - loss: 0.0030 - mae: 0.0198 - val_loss: 4.3637e-04 - val_mae: 0.0095
Epoch 5/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0030 - mae: 0.0195 - val_loss: 4.6063e-04 - val_mae: 0.0111
Epoch 6/200
15/15 [==============================] - 0s 26ms/step - loss: 0.0029 - mae: 0.0202 - val_loss: 4.2539e-04 - val_mae: 0.0050
Epoch 7/200
15/15 [==============================] - 0s 27ms/step - loss: 0.0030 - mae: 0.0193 - val_loss: 4.1951e-04 - val_mae: 0.0083
Epoch 8/200
15/15 [============================

### 9.2 Evaluación LSTM (métricas en escala original)


In [73]:
# ======== LSTM: desempeño en TRAIN ========

# Predicciones en ESCALA ESCALADA sobre el train
y_train_pred_lstm_scaled = model_lstm.predict(X_train_lstm).reshape(-1)
y_train_true_scaled      = y_train.reshape(-1)

# Volvemos a escala original
y_train_true_lstm = scaler.inverse_transform(
    y_train_true_scaled.reshape(-1, 1)
).reshape(-1)

y_train_pred_lstm = scaler.inverse_transform(
    y_train_pred_lstm_scaled.reshape(-1, 1)
).reshape(-1)

# Métricas en train
metrics_lstm_train = regression_metrics(y_train_true_lstm, y_train_pred_lstm)
print("LSTM – métricas TRAIN:", metrics_lstm_train)

# Gráfica histórico vs predicción en TRAIN
fig_train = plot_history_vs_pred(
    train_dates,
    y_train_true_lstm,
    y_train_pred_lstm,
    title="LSTM – Train vs predicción",
)
fig_train.show()

30/30 [==============================] - 2s 8ms/step
LSTM – métricas TRAIN: {'MAE': 35550173.44067762, 'RMSE': 82173844.54628095, 'MAPE': 18204.589200147475}


In [74]:
y_test_pred_lstm_scaled = model_lstm.predict(X_test_lstm).reshape(-1)
y_test_true_scaled = y_test.reshape(-1)

y_test_true_lstm = scaler.inverse_transform(y_test_true_scaled.reshape(-1, 1)).reshape(-1)
y_test_pred_lstm = scaler.inverse_transform(y_test_pred_lstm_scaled.reshape(-1, 1)).reshape(-1)

metrics_lstm = regression_metrics(y_test_true_lstm, y_test_pred_lstm)
print('LSTM – métricas test:', metrics_lstm)

fig = plot_history_vs_pred(test_dates, y_test_true_lstm, y_test_pred_lstm,
                           title='LSTM - Test vs predicción')
fig.show()

7/7 [==============================] - 0s 7ms/step
LSTM – métricas test: {'MAE': 15623194.433022691, 'RMSE': 20446429.391162917, 'MAPE': 1202.462344019155}


### 9.3 Pronóstico futuro con LSTM


In [75]:
N_FUTURE = 30  # días de pronóstico

# Última ventana del split completo (test) ya escalado
last_window_scaled = test_scaled[-WINDOW_SIZE:]

# Usamos el predict_fn especial para LSTM
future_scaled = recursive_forecast(
    model_lstm,
    last_window_scaled,
    N_FUTURE,
    predict_fn=lstm_predict_fn,  # 👈 aquí la magia
)

# Volvemos a escala original
future_preds = scaler.inverse_transform(future_scaled.reshape(-1, 1)).reshape(-1)

last_date = series.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1),
                             periods=N_FUTURE, freq='D')

fig = plot_future_forecast(
    series.index,
    series.values,
    future_dates,
    future_preds,
    title=f'Pronóstico LSTM {N_FUTURE} días – estado {STATE}',
)
fig.show()

1/1 [==============================] - 0s 25ms/step


## 10. Modelos Transformer

### 10.1 Model

In [76]:
# Datos en formato (batch, timesteps, features) para el Transformer
X_train_tf = X_train.reshape(-1, WINDOW_SIZE, 1)
X_val_tf   = X_val.reshape(-1, WINDOW_SIZE, 1)
X_test_tf  = X_test.reshape(-1, WINDOW_SIZE, 1)

import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    MultiHeadAttention,
    LayerNormalization,
    GlobalAveragePooling1D,
    Dense,
    Dropout,
)
from tensorflow.keras import Model

# ================== POSITIONAL ENCODING SINUSOIDAL (CONSTANTE) ==================

# d_model tiene que coincidir con la Dense de proyección
D_MODEL = 32

POS_ENCODING = tf.constant(
    get_sinusoidal_positional_encoding(WINDOW_SIZE, D_MODEL)
)  # shape (1, WINDOW_SIZE, D_MODEL)

# ================== MODELO TRANSFORMER ==================

inputs_tf = Input(shape=(WINDOW_SIZE, 1), name="transformer_input")

# Proyección inicial a d_model = D_MODEL
x = Dense(D_MODEL, name="proj_input")(inputs_tf)   # (batch, T, D_MODEL)

# Sumamos el positional encoding precomputado: shape (1, T, D_MODEL)
x = x + POS_ENCODING   # broadcasting a (batch, T, D_MODEL)

# === BLOQUE TRANSFORMER 1 (self-attention + feed-forward) ===

# Self-attention (8 cabezas, key_dim = D_MODEL)
attn1 = MultiHeadAttention(
    num_heads=4,
    key_dim=D_MODEL,
    name="mha_1"
)(x, x)
attn1 = Dropout(0.1, name="drop_attn_1")(attn1)

# Residual + norm
out1 = LayerNormalization(epsilon=1e-6, name="ln_attn_1")(x + attn1)

# Feed-forward interno: primero ampliamos, luego volvemos a D_MODEL
ff1 = Dense(2 * D_MODEL, activation="relu", name="ffn1_1")(out1)  # 256
ff1 = Dropout(0.1, name="drop_ffn_1")(ff1)
ff1 = Dense(D_MODEL, name="ffn1_2")(ff1)                          # 128

# Residual + norm (ahora las dims sí coinciden)
x = LayerNormalization(epsilon=1e-6, name="ln_ffn_1")(out1 + ff1)

# === FIN BLOQUE TRANSFORMER ===

# Pooling temporal
x = GlobalAveragePooling1D(name="gap")(x)

# Capas densas de salida
x = Dense(32, activation="relu", name="dense_out")(x)
outputs_tf = Dense(HORIZON, name="output")(x)

model_tf = Model(inputs=inputs_tf, outputs=outputs_tf, name="ts_transformer_simple")

model_tf.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="mse",
    metrics=["mae"],
)

model_tf.summary()


Model: "ts_transformer_simple"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 transformer_input (InputLayer)  [(None, 7, 1)]      0           []                               
                                                                                                  
 proj_input (Dense)             (None, 7, 32)        64          ['transformer_input[0][0]']      
                                                                                                  
 tf.__operators__.add_6 (TFOpLa  (None, 7, 32)       0           ['proj_input[0][0]']             
 mbda)                                                                                            
                                                                                                  
 mha_1 (MultiHeadAttention)     (None, 7, 32)        16800       ['tf.__operat

In [77]:
history_tf = model_tf.fit(
    X_train_tf,
    y_train,
    validation_data=(X_val_tf, y_val),
    epochs=200,
    batch_size=32,
    verbose=1,
)

Epoch 1/200
30/30 [==============================] - 2s 18ms/step - loss: 0.0845 - mae: 0.2245 - val_loss: 5.8282e-04 - val_mae: 0.0139
Epoch 2/200
30/30 [==============================] - 0s 8ms/step - loss: 0.0144 - mae: 0.0903 - val_loss: 0.0025 - val_mae: 0.0487
Epoch 3/200
30/30 [==============================] - 0s 9ms/step - loss: 0.0108 - mae: 0.0751 - val_loss: 8.4394e-04 - val_mae: 0.0247
Epoch 4/200
30/30 [==============================] - 0s 8ms/step - loss: 0.0093 - mae: 0.0696 - val_loss: 4.1036e-04 - val_mae: 0.0050
Epoch 5/200
30/30 [==============================] - 0s 9ms/step - loss: 0.0082 - mae: 0.0630 - val_loss: 5.2557e-04 - val_mae: 0.0117
Epoch 6/200
30/30 [==============================] - 0s 9ms/step - loss: 0.0070 - mae: 0.0576 - val_loss: 5.1031e-04 - val_mae: 0.0111
Epoch 7/200
30/30 [==============================] - 0s 9ms/step - loss: 0.0063 - mae: 0.0522 - val_loss: 7.6463e-04 - val_mae: 0.0192
Epoch 8/200
30/30 [==============================] - 0s 8m

### 10.2 Evaluación Transformer

In [78]:
# ======== Transformer: desempeño en TRAIN ========

# Predicciones en ESCALA ESCALADA sobre el train
y_train_pred_tf_scaled = model_tf.predict(X_train_tf).reshape(-1)
y_train_true_scaled    = y_train.reshape(-1)

# Volvemos a escala original
y_train_true_tf = scaler.inverse_transform(
    y_train_true_scaled.reshape(-1, 1)
).reshape(-1)

y_train_pred_tf = scaler.inverse_transform(
    y_train_pred_tf_scaled.reshape(-1, 1)
).reshape(-1)

# Métricas en train
metrics_tf_train = regression_metrics(y_train_true_tf, y_train_pred_tf)
print("Transformer - métricas TRAIN:", metrics_tf_train)

# Gráfica histórico vs predicción en TRAIN
fig_train_tf = plot_history_vs_pred(
    train_dates,
    y_train_true_tf,
    y_train_pred_tf,
    title="Transformer - Train vs predicción",
)
fig_train_tf.show()


30/30 [==============================] - 0s 4ms/step
Transformer - métricas TRAIN: {'MAE': 69458537.54214424, 'RMSE': 114656804.13832232, 'MAPE': 40626.31594927767}


In [79]:
# Predicciones en ESCALA ESCALADA
y_test_pred_tf_scaled = model_tf.predict(X_test_tf).reshape(-1)
y_test_true_scaled    = y_test.reshape(-1)

# Volver a escala original
y_test_true_tf = scaler.inverse_transform(
    y_test_true_scaled.reshape(-1, 1)
).reshape(-1)

y_test_pred_tf = scaler.inverse_transform(
    y_test_pred_tf_scaled.reshape(-1, 1)
).reshape(-1)

metrics_tf = regression_metrics(y_test_true_tf, y_test_pred_tf)
print("Transformer – métricas test:", metrics_tf)

fig = plot_history_vs_pred(
    test_dates,
    y_test_true_tf,
    y_test_pred_tf,
    title="Transformer - Test vs predicción",
)
fig.show()

7/7 [==============================] - 0s 3ms/step
Transformer – métricas test: {'MAE': 59204769.4741145, 'RMSE': 61168643.445904195, 'MAPE': 3446.558540282588}


### 10.3 Pronostico Futuro

In [80]:
N_FUTURE_TF = 30  # días de pronóstico con Transformer

last_window_scaled_tf = test_scaled[-WINDOW_SIZE:]

future_scaled_tf = recursive_forecast(
    model_tf,
    last_window_scaled_tf,
    N_FUTURE_TF,
    predict_fn=transformer_predict_fn,
)

future_preds_tf = scaler.inverse_transform(
    future_scaled_tf.reshape(-1, 1)
).reshape(-1)

last_date = series.index[-1]
future_dates_tf = pd.date_range(
    start=last_date + pd.Timedelta(days=1),
    periods=N_FUTURE_TF,
    freq="D",
)

fig = plot_future_forecast(
    series.index,
    series.values,
    future_dates_tf,
    future_preds_tf,
    title=f"Pronóstico Transformer {N_FUTURE_TF} días – estado {STATE}",
)
fig.show()


1/1 [==============================] - 0s 23ms/step


## 11. Predict all States

In [81]:
def lstm_predict_fn(model, X_2d: np.ndarray):
    """
    X_2d: shape (batch, window_size)
    Lo convertimos a (batch, window_size, 1) para el LSTM.
    """
    X_2d = np.asarray(X_2d)
    X_3d = X_2d.reshape(X_2d.shape[0], X_2d.shape[1], 1)
    return model.predict(X_3d)


def transformer_predict_fn(model, X_2d: np.ndarray):
    """
    X_2d: shape (batch, window_size)
    Lo convertimos a (batch, window_size, 1) para el Transformer.
    """
    X_2d = np.asarray(X_2d)
    X_3d = X_2d.reshape(X_2d.shape[0], X_2d.shape[1], 1)
    return model.predict(X_3d)

N_FUTURE = 30  # días hacia adelante

def forecast_state_future(series: pd.Series,
                          model,
                          scaler,
                          window_size: int,
                          n_future: int,
                          predict_fn) -> pd.DataFrame:
    """
    series: pd.Series con índice datetime y valores en escala ORIGINAL.
    model: modelo entrenado (LSTM o Transformer).
    scaler: el mismo scaler usado para entrenar en NY.
    predict_fn: lstm_predict_fn o transformer_predict_fn.
    """
    # Ordenar y limpiar
    series = series.dropna().sort_index()

    # Escalar con el mismo scaler del modelo
    series_scaled = scaler.transform(series.values.reshape(-1, 1)).reshape(-1)

    if len(series_scaled) < window_size:
        raise ValueError(f"Serie demasiado corta ({len(series_scaled)}) para window_size={window_size}")

    # Última ventana
    last_window_scaled = series_scaled[-window_size:]

    # Pronóstico recursivo en escala ESCALADA
    future_scaled = recursive_forecast(
        model,
        last_window_scaled,
        n_future,
        predict_fn=predict_fn,
    )

    # De vuelta a escala original
    future_values = scaler.inverse_transform(
        future_scaled.reshape(-1, 1)
    ).reshape(-1)

    # Fechas futuras
    last_date = series.index[-1]
    future_dates = pd.date_range(
        start=last_date + pd.Timedelta(days=1),
        periods=n_future,
        freq="D",
    )

    df_fore = pd.DataFrame({
        "date": future_dates,
        "y_pred": future_values,
    })
    return df_fore

### 11.1 Crear df_states

In [82]:
# Aseguramos tipo datetime y nos quedamos solo con lo necesario
df_states = (
    df_raw
    .copy()
    .loc[:, [STATE_COL, DATE_COL, TARGET_COL]]
)

# Si en df_raw hay varias filas por estado-fecha (por ejemplo, por condado),
# las agregamos. Puedes cambiar .sum() por .mean() si tiene más sentido.
df_states = (
    df_states
    .groupby([STATE_COL, DATE_COL], as_index=False)[TARGET_COL]
    .sum()
    .sort_values([STATE_COL, DATE_COL])
)

df_states.head()

,wwtp_jurisdiction,sample_collect_date,pcr_target_flowpop_lin
0,ak,2022-05-29,1.406782e+09
1,ak,2022-05-31,1.837388e+09
2,ak,2022-06-06,2.143289e+09
3,ak,2022-06-07,5.828170e+09
4,ak,2022-06-13,8.742622e+08


### 11.2 Predecir para todos los Estados

In [83]:
# Asegúrate que 'state' y 'date' sean las columnas correctas
all_states = sorted(df_states[STATE_COL].unique())
print("Total de estados:", len(all_states), all_states)

rows = []

for state in all_states:
    # Construir la serie del estado (índice datetime)
    df_state = (
        df_states[df_states[STATE_COL] == state]
        .sort_values(DATE_COL)
    )
    series_state = df_state.set_index(DATE_COL)[TARGET_COL]

    if len(series_state) < WINDOW_SIZE:
        print(f"Saltando estado {state}: solo {len(series_state)} puntos.")
        continue

    # Pronóstico con LSTM
    df_lstm = forecast_state_future(
        series_state,
        model=model_lstm,
        scaler=scaler,
        window_size=WINDOW_SIZE,
        n_future=N_FUTURE,
        predict_fn=lstm_predict_fn,
    )
    df_lstm.rename(columns={"y_pred": "y_lstm"}, inplace=True)

    # Pronóstico con Transformer
    df_tf = forecast_state_future(
        series_state,
        model=model_tf,
        scaler=scaler,
        window_size=WINDOW_SIZE,
        n_future=N_FUTURE,
        predict_fn=transformer_predict_fn,
    )
    df_tf.rename(columns={"y_pred": "y_tf"}, inplace=True)

    # Merge por fecha (deben ser iguales)
    df_merged = df_lstm.merge(df_tf, on="date")
    df_merged.insert(0, "state", state)

    rows.append(df_merged)

# DataFrame final con TODOS los estados
df_forecasts_all = pd.concat(rows, ignore_index=True)

df_forecasts_all.head()


Total de estados: 52 ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga', 'gu', 'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']
1/1 [==============================] - 0s 25ms/step


,state,date,y_lstm,y_tf
0,ak,2025-09-24,6.248407e+07,-8.401696e+06
1,ak,2025-09-25,5.677143e+07,1.307463e+07
2,ak,2025-09-26,5.387317e+07,-5.176841e+07
3,ak,2025-09-27,4.795546e+07,-4.384070e+07
4,ak,2025-09-28,3.446579e+07,-5.502439e+07


## 12. Matriz de adyacencia

### 12.1 Definicion

In [84]:
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga', 'gu',
          'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me', 'mi',
          'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm', 'nv', 'ny',
          'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'va', 'vt',
          'wa', 'wi', 'wv', 'wy']

neighbors = {
    "al": ["fl", "ga", "tn", "ms"],
    "ar": ["mo", "tn", "ms", "la", "tx", "ok"],
    "az": ["ca", "nv", "ut", "nm", "co"],
    "ca": ["or", "nv", "az"],
    "co": ["wy", "ne", "ks", "ok", "nm", "ut", "az"],
    "ct": ["ny", "ma", "ri"],
    "dc": ["md", "va"],
    "de": ["md", "pa", "nj"],
    "fl": ["al", "ga"],
    "ga": ["fl", "al", "tn", "nc", "sc"],
    "ia": ["mn", "wi", "il", "mo", "ne", "sd"],
    "id": ["wa", "or", "nv", "ut", "wy", "mt"],
    "il": ["wi", "ia", "mo", "ky", "in"],
    "in": ["mi", "oh", "ky", "il"],
    "ks": ["ne", "mo", "ok", "co"],
    "ky": ["il", "in", "oh", "wv", "va", "tn", "mo"],
    "la": ["tx", "ar", "ms"],
    "ma": ["ny", "vt", "nh", "ri", "ct"],
    "md": ["pa", "de", "va", "wv", "dc"],
    "me": ["nh"],
    "mi": ["oh", "in", "wi"],
    "mn": ["nd", "sd", "ia", "wi"],
    "mo": ["ia", "il", "ky", "tn", "ar", "ok", "ks", "ne"],
    "ms": ["tn", "al", "la", "ar"],
    "mt": ["nd", "sd", "wy", "id"],
    "nc": ["va", "tn", "ga", "sc"],
    "nd": ["mn", "sd", "mt"],
    "ne": ["sd", "ia", "mo", "ks", "co", "wy"],
    "nh": ["me", "ma", "vt"],
    "nj": ["ny", "pa", "de"],
    "nm": ["co", "ok", "tx", "az", "ut"],
    "nv": ["or", "id", "ut", "az", "ca"],
    "ny": ["pa", "nj", "ct", "ma", "vt"],
    "oh": ["mi", "in", "ky", "wv", "pa"],
    "ok": ["co", "ks", "mo", "ar", "tx", "nm"],
    "or": ["wa", "id", "nv", "ca"],
    "pa": ["ny", "nj", "de", "md", "wv", "oh"],
    "ri": ["ct", "ma"],
    "sc": ["nc", "ga"],
    "sd": ["nd", "mn", "ia", "ne", "wy", "mt"],
    "tn": ["ky", "va", "nc", "ga", "al", "ms", "ar", "mo"],
    "tx": ["nm", "ok", "ar", "la"],
    "ut": ["id", "wy", "co", "nm", "az", "nv"],
    "va": ["md", "dc", "wv", "ky", "tn", "nc"],
    "vt": ["ny", "ma", "nh"],
    "wa": ["id", "or"],
    "wi": ["mn", "ia", "il", "mi"],
    "wv": ["oh", "pa", "md", "va", "ky"],
    "wy": ["mt", "sd", "ne", "co", "ut", "id"],
    "ak": [],  # sin vecinos dentro de USA contiguo
    "hi": [],
    "gu": [],
}


### 12.3 Creacion

In [85]:
import pandas as pd
import numpy as np

# DataFrame de adyacencia lleno de ceros
adjacency = pd.DataFrame(
    0,
    index=states,
    columns=states,
    dtype=int
)

# Rellenar con 1 donde hay frontera
for s, neigh_list in neighbors.items():
    for t in neigh_list:
        if s in adjacency.index and t in adjacency.columns:
            adjacency.loc[s, t] = 1
            adjacency.loc[t, s] = 1  # simétrico

adjacency

,ak,al,ar,az,ca,co,ct,dc,de,fl,...,sd,tn,tx,ut,va,vt,wa,wi,wv,wy
ak,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
al,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
ar,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
az,0,0,0,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ca,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
co,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
ct,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
de,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fl,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
adjacency.loc["ny"][adjacency.loc["ny"] == 1]

ct    1
ma    1
nj    1
pa    1
vt    1
Name: ny, dtype: int32

## 13. Treshold de Riesgo

In [91]:
THRESHOLD = 100_000_000  # 50M

# Resumen por estado: máximo pronóstico LSTM / TF / combinado
state_summary = (
    df_forecasts_all
    .groupby("state")
    .agg(
        max_lstm=("y_lstm", "max"),
        max_tf=("y_tf", "max"),
    )
    .reset_index()
)

state_summary["max_any"] = state_summary[["max_lstm", "max_tf"]].max(axis=1)

# Flag de riesgo alto (2 = en pandemia, 0 = no)
state_summary["risk_flag"] = (state_summary["max_any"] > THRESHOLD).astype(int) * 2

state_summary.sort_values("max_any", ascending=False).head()

,state,max_lstm,max_tf,max_any,risk_flag
18,ky,1.262474e+09,3.045229e+09,3.045229e+09,2
48,wa,1.078915e+09,2.773996e+09,2.773996e+09,2
8,de,1.191438e+09,1.603467e+09,1.603467e+09,2
45,ut,1.300121e+09,1.425575e+09,1.425575e+09,2
49,wi,9.823086e+08,1.130667e+09,1.130667e+09,2


In [92]:
# Asegurar orden y alineación entre estado-resumen y la matriz de adyacencia
states_in_forecast = sorted(state_summary["state"].unique())

# Reindexamos la matriz de adyacencia a esos estados (por si alguno faltó)
adj = adjacency.reindex(index=states_in_forecast, columns=states_in_forecast).fillna(0).astype(int)

# Vector de riesgo (2 o 0) alineado al mismo orden
risk_vec = (
    state_summary.set_index("state")
    .loc[states_in_forecast, "risk_flag"]
    .astype(int)
)

# Empezamos con el vector de riesgo base (2 o 0)
risk_level = risk_vec.copy()

# Para cada estado en riesgo (2), marcamos sus vecinos directos como 1 (si aún son 0)
high_risk_states = risk_vec[risk_vec == 2].index

for s in high_risk_states:
    neighbors_s = adj.loc[s]
    neigh_indices = neighbors_s[neighbors_s == 1].index
    for t in neigh_indices:
        if risk_level[t] == 0:
            risk_level[t] = 1  # vecino de un estado en pandemia


In [93]:
# DataFrame por estado con métricas + nivel de riesgo 0/1/2
state_risk = (
    state_summary
    .set_index("state")
    .loc[states_in_forecast, ["max_lstm", "max_tf", "max_any"]]
    .assign(risk_level=risk_level)
    .reset_index()
)

state_risk.head()


,state,max_lstm,max_tf,max_any,risk_level
0,ak,7.288240e+07,1.307463e+07,7.288240e+07,0
1,al,7.109510e+07,-4.465229e+07,7.109510e+07,0
2,ar,8.022715e+07,6.241052e+07,8.022715e+07,0
3,az,1.890859e+08,1.145008e+08,1.890859e+08,2
4,ca,4.801498e+08,1.096819e+08,4.801498e+08,2


In [94]:
state_risk["risk_level"].value_counts()

risk_level
2    27
1    16
0     9
Name: count, dtype: int64

In [95]:
import plotly.express as px

# Copia y ajusta códigos a mayúsculas para el mapa de USA
df_map = state_risk.copy()
df_map["state_code"] = df_map["state"].str.upper()

fig_map = px.choropleth(
    df_map,
    locations="state_code",
    locationmode="USA-states",
    color="risk_level",
    scope="usa",
    hover_name="state_code",
    hover_data={
        "max_any": ":.2f",
        "max_lstm": ":.2f",
        "max_tf": ":.2f",
        "state_code": False,
    },
    labels={"risk_level": "Nivel de riesgo"},
)

fig_map.update_layout(
    title="Nivel de riesgo por estado (0 = sin riesgo, 1 = vecino, 2 = en pandemia)",
)

fig_map.show()

In [96]:
# Ordenamos por riesgo y por max_any para destacar los más críticos
df_bar = (
    state_risk
    .sort_values(["risk_level", "max_any"], ascending=[False, False])
    .head(15)  # top 15
)

fig_bar = px.bar(
    df_bar,
    x="state",
    y="max_any",
    color="risk_level",
    text="risk_level",
    labels={
        "state": "Estado",
        "max_any": "Máx. pronosticado (LSTM/TF)",
        "risk_level": "Nivel de riesgo",
    },
    title="Top estados por valor máximo pronosticado y nivel de riesgo",
)

fig_bar.update_traces(textposition="outside")
fig_bar.update_layout(xaxis_tickangle=-45)

fig_bar.show()


## 14. Colclusiones

En este trabajo se compararon dos enfoques de modelos no lineales para pronóstico de series de tiempo a nivel estatal: un modelo LSTM y un Transformer sencillo con self-attention y codificación posicional. En el escenario que planteé, el LSTM se comportó mejor que el Transformer, tanto en el conjunto de entrenamiento como en el de prueba.

Mi interpretación es que la dinámica de la serie está muy dominada por la dependencia local en el tiempo (lo que pasó en los últimos días) más que por patrones globales complejos en toda la ventana histórica. Los LSTM están diseñados justo para capturar ese tipo de memoria de corto y mediano plazo, mientras que el Transformer suele necesitar más datos, más features y más tuning. Aun así, el Transformer mostró potencial: con más capas, más señales (por ejemplo movilidad, clima, demografía) y una búsqueda sistemática de hiperparámetros, es muy probable que su desempeño mejore y se acerque o supere al LSTM.

También me quedaron varias líneas de trabajo abiertas que habrían hecho el análisis más robusto:

Aplicar pruebas estadísticas de comparación de modelos (por ejemplo algo tipo Diebold–Mariano) para no quedarme solo con la intuición visual y las métricas puntuales, sino validar formalmente si las diferencias entre LSTM y Transformer son significativas.

Usar la matriz de propagación / adyacencia no solo como herramienta de análisis de riesgo, sino incorporarla al entrenamiento como un peso adicional (por ejemplo un factor α que penalice más los errores en estados de alto riesgo o en sus vecinos). Esto se podría extender incluso a modelos espacio-temporales.

Profundizar en la definición del umbral de riesgo. Aquí usé un corte simple (150M en cualquiera de los modelos), pero sería interesante calibrarlo con datos históricos de olas pasadas, percentiles o métricas epidemiológicas más formales para decir “este nivel realmente implica que el estado está entrando en pandemia”.

Aun con estas limitaciones, el ejercicio muestra que este tipo de pipeline —descarga, limpieza, modelado con LSTM/Transformer y construcción de una matriz de riesgo/adyacencia— es una herramienta muy valiosa para la toma de decisiones. Si se implementara con datos actualizados de México, no solo serviría para analizar COVID-19, sino como base para vigilar la propagación de prácticamente cualquier enfermedad infecciosa a nivel estatal. Poder anticipar qué estados están en mayor riesgo y cuáles podrían verse afectados por contagio geográfico permitiría diseñar respuestas más rápidas y focalizadas (cuarentenas regionales, campañas de vacunación, uso dirigido de recursos hospitalarios, etc.).

En resumen, el LSTM resultó ser el modelo más efectivo en este escenario, pero el verdadero valor del proyecto está en la metodología completa: combinar modelos de series de tiempo con información espacial y criterios de riesgo para construir una visión más dinámica y accionable de la propagación de una pandemia.